In [1]:
import pandas as pd
data=pd.read_csv("FAOLEX.csv")
print(data.keys())

Index(['Record Id', 'Record URL', 'Document URL', 'Title', 'Original title',
       'Date of original text', 'Last amended date', 'Available website',
       'Language of document', 'Country/Territory', 'Regional organizations',
       'Territorial subdivision', 'Type of text', 'Repealed', 'Abstract',
       'Primary subjects', 'Domain', 'Keywords'],
      dtype='object')


In [2]:
import nltk
keywords=list(data["Keywords"])
country=list(data["Country/Territory"])
i=0
# Split alle keywords op i en liste og ændrer nan til at være en str med Nan
for words in keywords:
    if type(words)!=str:
        #print("type= ",type(words)," Index = ", i)
        words="Nan"
    if type(country[i])!=str:
        country[i]="NaN"
    keywords[i]=words.split(";")
    i+=1



In [3]:
i=0

for words in keywords:
    keywords[i]=words + [" "+country[i]]
    i+=1

In [5]:
# prints keywords for a single text
print(keywords[0])

['basic legislation', ' marine fisheries', ' navigation', ' foreign fishing', ' access right', ' fishing authorization', ' fishing gear/fishing method', ' offences/penalties', ' research', ' non-commercial fishing', ' concession', ' enforcement/compliance', ' fishing vessel', ' marking/identification', ' fishery management and conservation', ' Algeria']


In [6]:
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import numpy as np
stemmer = SnowballStemmer("english")
stemmed_words=[]

# Stem keywords
for i in range(len(keywords)):
    stemmed_words.append([stemmer.stem(word) for word in keywords[i]])



In [7]:
print(keywords[14083])
print(stemmed_words[14083])

['basic legislation', ' land reform', ' land tenure', ' transfer\xa0rights\xa0', ' public land', ' land subdivision', ' right of use', ' expropriation', ' lease', ' common property', ' ownership', ' inheritance', ' cadastre/land registration', ' institution', ' agricultural land', ' land based credit', ' vested rights\xa0', ' farming', ' agricultural development', ' cooperative/producer organization', ' legal proceedings/administrative proceedings', ' Russian Federation']
['basic legisl', ' land reform', ' land tenur', ' transfer\xa0rights\xa0', ' public land', ' land subdivis', ' right of us', ' expropri', ' leas', ' common properti', ' ownership', ' inherit', ' cadastre/land registr', ' institut', ' agricultural land', ' land based credit', ' vested rights\xa0', ' farm', ' agricultural develop', ' cooperative/producer organ', ' legal proceedings/administrative proceed', ' russian feder']


In [8]:
# Find number of extinct domains
a= list(set(data["Repealed"]))
i=0
for words in a:
    if type(words)!=str:
        #print("type= ",type(words)," Index = ", i)
        words="Nan"
    a[i]=words.split(";")

    i+=1



In [9]:
# print different domains
a=sum(a,[])
i=0
for word in a:
        word=word.strip()
        a[i]=word
        i+=1

print(set(a))




{'Y', 'Nan'}


In [11]:
# Count number documents in a language
languages=list(data["Language of document"])
print(languages.count("English"))

53564


In [12]:
# Make a top2vec topic model
from top2vec import Top2Vec
i=0
for word in keywords:
    keywords[i]="".join(word)
    i+=1
model = Top2Vec(keywords,embedding_model="doc2vec")


2023-05-29 10:18:19,793 - top2vec - INFO - Pre-processing documents for training
2023-05-29 10:18:26,880 - top2vec - INFO - Creating joint document/word embedding
2023-05-29 10:21:42,433 - top2vec - INFO - Creating lower dimension embedding of documents
2023-05-29 10:22:53,559 - top2vec - INFO - Finding dense areas of documents
2023-05-29 10:23:01,022 - top2vec - INFO - Finding topics


In [13]:
topic_sizes, topic_nums = model.get_topic_sizes()
topic_sizes

array([1045,  999,  867, ...,   18,   17,   17])

In [14]:
topic_words,word_scores,topic_nums = model.get_topics(10)

# Topic words: Ord i en given topic
#Words_score: Hvor 

In [184]:
print(topic_words)

[['great' 'kingdom' 'ireland' 'britain' 'northern' 'jersey' 'united'
  'swine' 'slaughtering' 'equine' 'animal' 'disinfection' 'feedstuffs'
  'steroids' 'hormones' 'goats' 'feed' 'sheep' 'disinfestation' 'meat'
  'cattle' 'limit' 'vaccination' 'camelids' 'drugs' 'inspection'
  'welfare' 'guernsey' 'poultry' 'reproduction' 'veterinary' 'herd'
  'svalbard' 'additives' 'rodents' 'traceability' 'mrl' 'maximum' 'pests'
  'diseases' 'quarantine' 'health' 'residue' 'book' 'edible' 'birds'
  'treated' 'residues' 'procedures' 'is']
 ['port' 'transshipment' 'unreported' 'illegal' 'unregulated' 'iuu'
  'foreign' 'landing' 'fishing' 'high' 'state' 'seas' 'bycatch'
  'migratory' 'marine' 'cartilaginous' 'nan' 'vessel' 'fisheries'
  'fishery' 'harbour' 'svalbard' 'catch' 'territorial' 'exclusive'
  'fishes' 'allowable' 'eez' 'economic' 'marshall' 'artisanal' 'palau'
  'seychelles' 'size' 'method' 'mammals' 'total' 'gear' 'enforcement'
  'charge' 'sea' 'vanuatu' 'maritime' 'kiribati' 'zone' 'federate

In [186]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=2, num_docs=5)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 75345, Score: 0.949529230594635
-----------
fishery management and conservation marine fisheries territorial sea EEZ-Exclusive Economic Zone maritime zone sovereignty Peru
-----------

Document: 74771, Score: 0.9338657259941101
-----------
marine fisheries EEZ-Exclusive Economic Zone maritime zone territorial sea continental shelf data collection/reporting EEZ-Exclusive Economic Zone maritime zone territorial sea continental shelf Russian Federation
-----------

Document: 43109, Score: 0.932529866695404
-----------
authorization/permit territorial sea EEZ-Exclusive Economic Zone fishery management and conservation fishing area Norway
-----------

Document: 36836, Score: 0.9314826726913452
-----------
EEZ-Exclusive Economic Zone territorial sea continental shelf marine fisheries marine fishes research fishing vessel allocation/quota fishing gear/fishing method fishing authorization fishery management and conservation Russian Federation
-----------

Document: 74773, Score: 0.92

In [195]:
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["denmark"], num_topics=3)
print(topic_words)
print(topic_nums)

[array(['greenland', 'allocation', 'catch', 'fishing', 'allowable',
       'fishery', 'bycatch', 'quota', 'fishes', 'crustaceans', 'marine',
       'total', 'size', 'cartilaginous', 'mesh', 'fisheries', 'svalbard',
       'seasons', 'gear', 'diadromous', 'method', 'vessel', 'landing',
       'molluscs', 'prohibited', 'commercial', 'denmark', 'turtles',
       'transshipment', 'charge', 'migratory', 'iceland', 'mammals',
       'area', 'foreign', 'norway', 'territorial', 'port', 'conservation',
       'seas', 'sea', 'exclusive', 'high', 'fee', 'eez', 'harbour', 'is',
       'jurisdiction', 'continental', 'shelf'], dtype='<U15'), array(['qatar', 'bahrain', 'kuwait', 'nuclear', 'oman', 'sweden',
       'arabia', 'singapore', 'procedural', 'emirates', 'matters',
       'natural', 'iraq', 'greece', 'saudi', 'energy', 'gas', 'denmark',
       'coral', 'hydropower', 'latvia', 'radiation', 'coal', 'palestine',
       'aquatic', 'israel', 'prohibited', 'fee', 'north', 'sinking',
       'guam', 

In [196]:
topic_mapping = model.hierarchical_topic_reduction(num_topics=20)

In [197]:
print(model.topic_words_reduced)

[['steroids' 'hormones' 'mrl' 'residue' 'drugs' 'limit' 'hygiene'
  'maximum' 'swine' 'procedures' 'residues' 'sanitary' 'feed'
  'feedstuffs' 'veterinary' 'animal' 'toxicity' 'poultry' 'poisoning'
  'equine' 'slaughtering' 'meat' 'cattle' 'sheep' 'vaccination' 'goats'
  'additives' 'health' 'dairy' 'mercosur' 'safety' 'disinfestation'
  'codex' 'inspection' 'rodents' 'treated' 'haccp' 'disinfection'
  'andean' 'edible' 'packaging' 'pests' 'traceability' 'trade'
  'antimicrobial' 'diseases' 'welfare' 'labelling' 'milk' 'alimentarius']
 ['equine' 'sheep' 'swine' 'goats' 'cattle' 'rodents' 'herd' 'camelids'
  'book' 'reproduction' 'poultry' 'animal' 'slaughtering' 'vaccination'
  'mercosur' 'animals' 'welfare' 'meat' 'health' 'disinfestation' 'birds'
  'jersey' 'hygiene' 'sericulture' 'apiculture' 'steroids' 'hormones'
  'drugs' 'procedures' 'disinfection' 'quarantine' 'identification'
  'north' 'pests' 'feed' 'sanitary' 'ranching' 'inspection' 'marking'
  'captive' 'diseases' 'edible' '

In [202]:
model.topic_words_reduced[4]

array(['viticulture', 'oenological', 'tribunal', 'court', 'herzegovina',
       'beverages', 'tobacco', 'geographical', 'bosnia', 'indication',
       'fraud', 'settlement', 'codex', 'inspection', 'lao',
       'compensation', 'dispute', 'intellectual', 'labelling', 'trade',
       'certification', 'justice', 'practices', 'consumer', 'lanka',
       'liability', 'packaging', 'deceit', 'information', 'plant',
       'haccp', 'traceability', 'alimentarius', 'offences', 'internal',
       'tristan', 'adulteration', 'classification', 'singapore',
       'breeders', 'access', 'seeds', 'commodities', 'vested', 'sugar',
       'floriculture', 'veterinary', 'liechtenstein', 'legal',
       'agricultural'], dtype='<U14')

In [82]:
topic_words,word_scores,topic_nums = model.get_topics(10)
topic_words

array([['mesh', 'fishing', 'seasons', 'size', 'fishery', 'method',
        'cartilaginous', 'catch', 'bycatch', 'fisheries', 'landing',
        'prohibited', 'marine', 'total', 'allowable', 'charge',
        'allocation', 'fishes', 'vessel', 'gear', 'quota', 'artisanal',
        'foreign', 'commercial', 'diadromous', 'port', 'migratory',
        'transshipment', 'turtles', 'mammals', 'molluscs', 'crustaceans',
        'illegal', 'high', 'seas', 'inland', 'iuu', 'territorial',
        'unregulated', 'area', 'exclusive', 'state', 'eez',
        'enhancement', 'management', 'conservation', 'coral',
        'unreported', 'repopulation', 'sea'],
       ['floriculture', 'planting', 'seeds', 'material', 'fibre',
        'plant', 'cane', 'beet', 'variety', 'vegetables', 'breeders',
        'oleaginous', 'legumes', 'fruits', 'edible', 'nuts', 'sugar',
        'certification', 'rice', 'tobacco', 'cereals', 'labelling',
        'weeds', 'crops', 'oenological', 'production', 'packaging',
        '

In [274]:
# Tæl hvor mange der er repelaed inden for et dato interval
dates=data["Date of original text"]
repealed=data["Repealed"]
antal=0
for i in range(1800,1970):
    c=dates==str(i)
    d=repealed=="Y"
    antal+=sum(c*d)
print(antal)


91


In [ ]:
model.generate_topic_wordcloud(2)
# 167 kunne godt være fødevare og børn